In [1]:
import lincs
import liblincs

In [2]:
def walk(output_file, node_name, node, parent_node=None, depth=0):
    indent = "    " * depth
    node_type = type(node)
    node_type_name = node_type.__name__

    # Don't show objects of these types
    if node_type_name in [
        "ModuleSpec", "SourceFileLoader", "ExtensionFileLoader",
    ]:
        return
    
    # Don't show special attributes
    if node_name in [
        "__dict__", "__class__", "__dir__", "__getnewargs__", "__slots__", "__instance_size__",
        "__setattr__", "__getattr__", "__delattr__", "__getattribute__",
        "__weakref__",
        "__str__", "__repr__", "__doc__",
        "__hash__", "__format__",
        "__eq__", "__ge__", "__gt__", "__le__", "__lt__", "__ne__", "__abs__", "__add__", "__and__", "__bool__", "__ceil__", "__divmod__", "__float__", "__floor__", "__floordiv__", "__index__", "__int__", "__invert__", "__lshift__", "__mod__", "__mul__", "__neg__", "__or__", "__pos__", "__pow__", "__radd__", "__rand__", "__rdivmod__", "__reduce__", "__rfloordiv__", "__rlshift__", "__rmod__", "__rmul__", "__ror__", "__round__", "__rpow__", "__rrshift__", "__rshift__", "__rsub__", "__rtruediv__", "__rxor__", "__sub__", "__truediv__", "__trunc__", "__xor__",
        "__init_subclass__", "__reduce_ex__", "__cause__", "__context__", "__module__", "__new__", "__sizeof__", "__subclasshook__", "__suppress_context__", "__traceback__",
    ]:
        return

    # Defensive: never recurse too deep
    if depth > 4:
        output_file.write(f"{indent} {node_name}: {node_type_name}  UNFINISHED\n")
        return

    output_file.write(f"{indent}{node_name}\n")

    # Don't recurse into our enums: enum values have an attribute pointing to its type
    if node_type == parent_node:
        return

    # Don't recurse into special attributes
    if node_name.startswith("__") and node_name.endswith("__"):
        return

    # Don't recurse into objects of these types
    if node_type_name in [
        "wrapper_descriptor", "getset_descriptor", "method_descriptor", "member_descriptor",
        "property", "function", "method", "builtin_function_or_method",
        "str", "bool", "int", "float", "NoneType",
        "list", "dict", "tuple",
    ]:
        return

    for child_name in sorted(dir(node)):
        try:
            child_node = getattr(node, child_name)
        except AttributeError:
            # Don't get stuck on attributes listed in 'dir' but not actually there
            pass
        else:
            walk(output_file, child_name, child_node, node, depth + 1)


with open("lincs.txt", "w") as f:
    walk(f, "lincs", lincs)


with open("liblincs.txt", "w") as f:
    walk(f, "liblincs", liblincs)